# Compréhension du problème

L'objectif, sur la plateforme kaggle, du concours "Costa Rican Household Poverty Level Prediction" est de développer un modèle de prédiction du niveau de pauvereté d'un foyer en s'appuyant d'une part sur les caractéristiques individuelles de ces membres et d'autre part des caractéristiques du foyer.

Les données de ce concours consistent en deux fichiers : train.csv et test.csv.

Ces fichiers contiennent respectivement 9557 et 23856 observations.

Les deux fichiers contiennent 142 variables commununes. Ces variables peuvent être soit de niveau individu soit du niveau foyer.

Le fichier d'apprentissage "train.csv" dispose d'une variable en plus : "Target". Cette dernière correspond à la variable à prédire.

La définition des 143 variables est disponible sur le site kaggle : Costa Rican Household Poverty Level Prediction.


La variable "Target" peut prendre une des 4 modalités : 1 à 4. La valeur 1 correspond au niveau extrême de pauvereté et le niveau 4 au niveau "non vulnérable".

1 = Extrême
2 = Modérée 
3 = Vulnérable
4 = Non vulnérable

On est donc face à un problème :
    - d'apprentissage supervisé
    - Et de classification multi-classe


## Objectifs

L'objectif est de prédire le niveau de pauvereté au niveau de chaque foyer. 

Les observations sont de niveau individu. Les variables peuvent être soit de niveau individu soit de niveau foyer.

Chaque individu dispose d'un identifiant unique (id).

Chaque foyer est caractérisé par un identifant unique (idhogar) et un "Représentant du foyer" (parentesco1 = 1).

Pour la prédiction :
- nous serons amené à agréger les variables de niveau individu pour créer des variables de niveau foyer.
- A faire de prédiction au niveau de chaque individu. Cependant pour le scoring, la seule prédiction prise en compte pour un foyer est celle de son représentant.




## Métrique

La métrique proposée dans le cadre de ce concours est le Macro F1-Score.

Il s'agit donc d'une moyenne du F1-score de chaque classe.

à compléter

## Méthodologie

Le but utlime de ce projet est de prédire le niveau de pauvereté de chaque foyer.
Cependant avant de tester différents modèles, il est important de comprendre la problématique et les données.

La méthodologie suivie pour modèliser ce niveau de pauvereté est la suivante :
    
    - Compréhension du problème
    - Analyse exploratoire des données
    - Préparation des données / variables pour le modèle (Feature engineering)
    - Modèlisation
    - Choix du modèle
    - Conclusion

# Chargement des modules / fonctions et Import des données

## Chargement des modules

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Affichage Image
from IPython.display import Image
from IPython.display import display

# MàJ de certains paramétres (Affichage et manipulation des données)
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 18
plt.rcParams['patch.edgecolor'] = 'k'


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


#plt.style.use('seaborn-whitegrid')
sns.set(style = 'darkgrid')

# Outils de modélisation
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline


from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb


# Macro F1 score
scorer = make_scorer(f1_score, greater_is_better=True, average = 'macro')


# Fonctions de réequilibrage des données
from imblearn.over_sampling import SMOTE

## Import des données

In [ ]:
#Le path vers les fichiers de données en local
#filepath ='/Users/moi/Documents/CoursDauphine/Module3/supervised/costa-rican-household-poverty-prediction'
#filepath_train = '/Users/moi/Documents/CoursDauphine/Module3/supervised/costa-rican-household-poverty-prediction/train.csv'
#filepath_test = '/Users/moi/Documents/CoursDauphine/Module3/supervised/costa-rican-household-poverty-prediction/test.csv'

#filepath_submission = '/Users/moi/Documents/CoursDauphine/Module3/supervised/costa-rican-household-poverty-prediction/sample_submission.csv' 

In [ ]:
#df_train = pd.read_csv(filepath_train, sep=',', decimal='.')
#df_test = pd.read_csv(filepath_test, sep=',', decimal='.')
#df_submit = pd.read_csv(filepath_submission, sep=',', decimal='.')

In [ ]:
df_train = pd.read_csv('../input/train.csv', sep=',', decimal='.')
df_test = pd.read_csv('../input/test.csv', sep=',', decimal='.')
df_submit = pd.read_csv('../input/sample_submission.csv', sep=',', decimal='.')

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_submit.shape

# Analyse exploratoire des données

## Analyse des variables explicatives

In [ ]:
df_train.head()

In [ ]:
 df_test.head()

Les deux fichiers de données contiennent plusieurs types de variables :
    - Booléen (catégorie)
    - Chaîne de caractères (identifiants)
    - décimal (v2a1 : Montant du loyer )

La commande df_train.info() permet d'afficher une vue globale des types de données du dataframe : 

In [ ]:
df_train.info()

In [ ]:
df_test.info()

Le jeu de données d'apprentissage contient :
- 130 variables de types integer. Ces variables représentent des variables de type "Category" ou "Ordinal"
- 8 variables de types float. Ces variables correspondent à des montants et/ou à des taux.
- 5 variables de type object. Une partie de ces variables représentent des identifiants qui ne seraient pas utiles pour la partie modèlisation. Cependant, l'autre partie doit être analysée et étudiées. 

Etudions la distribution de chaque type de variable.

Le jeux de données de test contient une variable en moins de type integer. Cette dernière correspond à la variable à prédire (Target).

*Etude des différents types de variables du jeux de données d'apprentissage*

**Variables de type "Integer"**

Etudions la distribution des variables de type integer :

In [ ]:
df_train.select_dtypes(include=['int64']).nunique().value_counts().sort_index().plot.bar(color = 'blue', figsize = (8,6), edgecolor ='k', linewidth = 2)

plt.xlabel('Nombre de valeurs uniques prise par la variables'); plt.ylabel('Nombre de colonnes');
plt.title('Nombre de colonnes de type Integer avec ces valeurs uniques');

Les variables avec deux valeurs uniques représentent des booléens (0 ou 1). La plupart des variables sont de type booléen.

**Variables de type "Float"**

Etudions la distribution des variables de type "float" en fonction du niveau de pauvereté.

In [ ]:
from collections import OrderedDict

plt.figure(figsize = (20, 16))
plt.style.use('fivethirtyeight')

# Color mapping
colors = OrderedDict({1: 'red', 2: 'orange', 3: 'blue', 4: 'green'})
poverty_mapping = OrderedDict({1: 'extreme', 2: 'moderate', 3: 'vulnerable', 4: 'non vulnerable'})

for i,col in enumerate(df_train.select_dtypes(include=['float64'])):

    ax = plt.subplot(6, 2, i + 1)
    # Iterate through the poverty levels
    for poverty_level, color in colors.items():
        # Plot each poverty level as a separate line
        sns.kdeplot(df_train.loc[df_train['Target'] == poverty_level, col].dropna(), 
                    ax = ax, color = color, label = poverty_mapping[poverty_level])
        
    plt.title(f'{col.capitalize()} Distribution'); plt.xlabel(f'{col}'); plt.ylabel('Density')


plt.subplots_adjust(top = 2)

A priori les variables Niveau moyen d'éducation("meaneduc") et niveau de pauvereté de l'individu("Target") sont liées.
En effet, plus le niveau moyen d'éducation des adultes dans le foyer est elevé plus le niveau de pauvereté est sévère (donc le degré de pauvereté est moins sévère).

Le théme de l'éducation est potentiellement un des axes influent dans la calcul du niveau de pauvereté d'un foyer. 

**Variables de type "object"**

In [ ]:
df_train.select_dtypes(include = ['object']).head()

Les deux attributs Id et idhogar permettent d'identifier respectivement un individu et un foyer.
Ces deux attributs ne seront pas utilisés dans les modèles. Ils resteront de type "object".

Les 3 attributs dependency, edjefe et edjefa contiennent à la fois des valeurs numériques et de deux autres valeurs de type caractères "yes" et "no".

D'après la définition des variables, le "yes" et le "no" correspondent respectivement à 1 et 0.

Nous allons donc remplacer ces deux valeurs des les deux jeux de données df_train et df_test.

Nous allons donc commencer par concaténer les deux jeux de données

In [ ]:
# Merge des deux dataframe train et test et ajout de la colonne Target aux données de test
df_test['Target'] = np.nan
data = df_train.append(df_test, ignore_index = True)

In [ ]:
data.shape

In [ ]:
data['Target'].isnull().sum()

In [ ]:
map_dict = {'yes':1,'no':0}

data["dependency"] = data["dependency"].replace(map_dict).astype('float64')
data["edjefe"] = data["edjefe"].replace(map_dict).astype('float64')
data["edjefa"] = data["edjefa"].replace(map_dict).astype('float64')

In [ ]:
data["dependency"] = data["dependency"].astype('float64')
data["edjefe"] = data["edjefe"].astype('float64')
data["edjefa"] = data["edjefa"].astype('float64')

## Etude des incohérences dans le jeu de données

Certains individus du même foyer ont des labels (Target) différents. Il s'agit d'une anomalie dans le jeu de données.
Le label à considérer est celui du l'individu tête de foyer.

In [ ]:
data[data["Target"].notnull()].groupby("idhogar")["Target"].nunique().value_counts()

Il y a 85 individus avec un label différent de celui de l'individu tête de foyer.

In [ ]:
# Liste des individus concernés
data_multi_target = data[data["Target"].notnull()].groupby("idhogar")["Target"].apply(lambda x : x.nunique()==1)

data_multi_target = data_multi_target[data_multi_target != True]

In [ ]:
data_multi_target.shape

In [ ]:
data_multi_target.index[1]

Ci-dessous un exemple de foyer multi label :

In [ ]:
data[data["idhogar"] == data_multi_target.index[0]][['idhogar', 'parentesco1', 'Target']]

In [ ]:
for id in data_multi_target.index:
    True_label = data[(data['idhogar'] == id) & ( data['parentesco1'] == 1)]["Target"] 
    data.loc[data['idhogar'] == id, "Target"]  = np.where(True_label.notnull(), True_label, data.loc[data['idhogar'] == id, "Target"])

In [ ]:
data['Target'].isnull().sum()

## Etude des données manquantes

In [ ]:
# Number of missing in each column
missing = pd.DataFrame(data.isnull().sum()).rename(columns = {0: 'total'})

# Create a percentage missing
missing['percent'] = missing['total'] / len(data)

missing.sort_values('percent', ascending = False).head(10).drop('Target')

Les colonnes avec des données manquantes :
- v2a1    : loyer mensuel
- v18q1   : Nbre de tablette ds la maison
- rez_esc :  Years behind in school
- meaneduc :  Average years of education for adults
- SQBmeaned : square of the mean years of education of adults (>=18) in the household

*Variable v18q1*

Cette variable est liée à la variable de type individu v18q. Cette dernière indique si l'individu dispose d'une tablette ou pas.
v18q ne contient de valeurs manquantes. Nous allons donc déduire la variable au niveau foyer v18q1 à partir de la variable de type individu.

In [ ]:
# Distribution des nombres de tablettes dans le foyer
data[data['parentesco1'] == 1]['v18q1'].value_counts().sort_index().plot.bar( figsize = (8,6),edgecolor ='k', linewidth = 2)
# Formatting

In [ ]:
data[data['parentesco1'] == 1]['v18q1'].value_counts()

In [ ]:
# Répartition de la variable de type individu par rapport à la variable de type foyer
data[data['parentesco1'] == 1].groupby('v18q1')['v18q'].sum()

On en déduit des deux répartitions ci-dessus que les valeurs manquantes correspondent à 0

In [ ]:
data['v18q1'] = data['v18q1'].fillna(0)

*Variable v2a1*

Cette variable correspond au montant du loyer.
Elle est non renseignée pour 24263 observations.

Les variables ci-dessous permettent de distinguer les locataires, des propriétaires.
Il serait donc intéresser d'étudier la distribution de ces variables.

Si la variable "tipovivi1" est égale à 1, la famille est propriétaire de l'appartement et n'a plus de crédit dessus. Le loyer est égale donc à 0

- tipovivi1 =1 own and fully paid house
- tipovivi2 =1 own,  paying in installments"
- tipovivi3 =1 rented
- tipovivi4 =1 precarious
- tipovivi5 =1 other(assigned,  borrowed)"

In [ ]:
tipovivi = data.columns.str.startswith('tipovivi')

In [ ]:
col = ["v2a1","tipovivi1","tipovivi2","tipovivi3","tipovivi4","tipovivi5"]
data.loc[(data["v2a1"].isnull() ) & (data["parentesco1"] ==1),col].sum().plot.bar()

La majorité des reponsable de foyer n'ayant de loyer renseigné sont propriétaire de leurs maisons.

Pour les autres situations, nous ne disposons pas d'assez de données pour en déduire la cause.

Pour les maisons dont le responsable du foyer est propriétaire, le montant du loyer sera alimenté avec 0 pour les autres nous allons les topées "Données manquantes".

In [ ]:
data.loc[(data['tipovivi1'] == 1 & data['v2a1'].isnull() ), 'v2a1'] = 0

In [ ]:
# Pour les autres valeurs manquantes et vu le manque d'informations, on propose d'imputer par la médiane (Voir partie modèle)

data.loc[data['v2a1'].isnull(), 'v2a1'] = 0

*Variable rez_esc*

Cette variable correspond au nombre d'années en dehors de l'école.
Cette varaible n'a de sens que pour les individus âgés de 7 à 19 ans.
Si l'âge de l'individu est en dehors de cet intervalle, cette variable vaut 0 (Voir descritpion de la variable + compléments d'informations dans le forum).

In [ ]:
# On vérifie s'il n y a pas des incohérences entre l'âge et cette variable.
data[data['rez_esc'].notnull()]['age'].describe()

L'intervalle d'âge pour les données sourcées est cohérent (entre 7 et 19 ans).

In [ ]:
# Imputation des observation ayant un âge en dehors de l'intervalle 7 à 19 ans
data.loc[((data['age'] > 19) | (data['age'] < 7)) & (data['rez_esc'].isnull()) ,'rez_esc'] = 0

In [ ]:
# La valeur ma maximale prise par cette variable est 5. Or certaines obsevations ont des valeurs supérieurs.

data['rez_esc'].unique()

In [ ]:
# On va donc remplacer toute valeur supérieur à 5 par le max

data.loc[data['rez_esc'] > 5, 'rez_esc']  = 5

In [ ]:
# Pour les autres valeurs observations et vu le manque d'informations, on propose d'imputer par la médiane (Voir partie modèle)
data.loc[data['rez_esc'].isnull(),'rez_esc' ] = 0

*Variable meaneduc*

In [ ]:
# Au vu du nombre d'observations concernées, on impute avec 0
data.loc[data['meaneduc'].isnull(),'meaneduc' ] = 0


*Variable SQBmeaned*

In [ ]:
# Pour être cohérent avec l'imputation ci-dessus, on impute avec 0
data.loc[data['SQBmeaned'].isnull(), 'SQBmeaned'] = 0

In [ ]:
data.isnull().sum()

## Etude des interractions entre les variables

Le dataframe head ci-dessous contient uniquement les individus responsable de chaque foyer.

In [ ]:
head = df_train.loc[(data["parentesco1"] ==1) & (data["Target"].notnull())].copy()

*Distribution de la variable Target*

In [ ]:
distrib_target = head['Target'].value_counts().reset_index().rename(columns = {'index' : 'level' })

In [ ]:
distrib_target['Household_type'] = distrib_target['level'].map(poverty_mapping)

In [ ]:
distrib_target.head()

In [ ]:
sns.set(style = 'whitegrid', font_scale=1.4)
fig = plt.subplots(figsize=(15, 8))
ax = sns.barplot(x = 'Household_type', y = 'Target', data = distrib_target, palette='Accent', ci = None).set_title('Distribution de la pauvereté')

Le jeux de données n'est pas équilibré. Les modèles auront du mal à prédire la classe minoritaire("Extreme").
Nous allons par la suite étudier des méthodes pour ré-équilibrer ce jeu de données.

## Future engineering

L'objectif de ce projet est de prévoir le niveau de pauvereté au niveau de chaque foyer.
Il convient, dans le cadre de cette section, d'agréger les données au niveau individu et de créer de nouvelles variables par foyer.

Nous allons commencer par distinguer deux types de variables :
 - Variables de niveau individu
 - Varibales de niveau foyer

Les variables de type individu seront agrégées et retraitées pour en extraire de nouvelles variables de type foyer.

*Identifiants*

In [ ]:
ids = ['Id', 'idhogar']
target = ['Target']

*Variables de niveau individu*

In [ ]:
ind_bool = ['v18q', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 
            'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
            'parentesco1', 'parentesco2',  'parentesco3', 'parentesco4', 'parentesco5', 
            'parentesco6', 'parentesco7', 'parentesco8',  'parentesco9', 'parentesco10', 
            'parentesco11', 'parentesco12', 'instlevel1', 'instlevel2', 'instlevel3', 
            'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 
            'instlevel9', 'mobilephone']

ind_ordered = ['rez_esc', 'escolari', 'age']

*Variables de niveau foyer*

In [ ]:
hh_bool = ['hacdor', 'hacapo', 'v14a', 'refrig', 'paredblolad', 'paredzocalo', 
           'paredpreb','pisocemento', 'pareddes', 'paredmad',
           'paredzinc', 'paredfibras', 'paredother', 'pisomoscer', 'pisoother', 
           'pisonatur', 'pisonotiene', 'pisomadera',
           'techozinc', 'techoentrepiso', 'techocane', 'techootro', 'cielorazo', 
           'abastaguadentro', 'abastaguafuera', 'abastaguano',
            'public', 'planpri', 'noelec', 'coopele', 'sanitario1', 
           'sanitario2', 'sanitario3', 'sanitario5',   'sanitario6',
           'energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4', 
           'elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 
           'elimbasu5', 'elimbasu6', 'epared1', 'epared2', 'epared3',
           'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 
           'tipovivi1', 'tipovivi2', 'tipovivi3', 'tipovivi4', 'tipovivi5', 
           'computer', 'television', 'lugar1', 'lugar2', 'lugar3',
           'lugar4', 'lugar5', 'lugar6', 'area1', 'area2']

hh_ordered = [ 'rooms', 'r4h1', 'r4h2', 'r4h3', 'r4m1','r4m2','r4m3', 'r4t1',  'r4t2', 
              'r4t3', 'v18q1', 'tamhog','tamviv','hhsize','hogar_nin',
              'hogar_adul','hogar_mayor','hogar_total',  'bedrooms', 'qmobilephone']

hh_cont = ['v2a1', 'dependency', 'edjefe', 'edjefa', 'meaneduc', 'overcrowding']
           
           

*Variables d'ordre 2 (squared)*

In [ ]:
sqr_ = ['SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 
        'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned', 'agesq']

### Analyse de niveau foyer

In [ ]:
head = data.loc[(data["parentesco1"] ==1), ids + hh_bool + hh_ordered + hh_cont + target].copy()

In [ ]:
head.head()

In [ ]:
head.shape

*Corrélation entre les variables*

In [ ]:
# Matrice de corrélation
corr_matrix = head.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.95)]

to_drop

Plusieurs variables liées à la taille du foyer et à l'âge sont très largement corrélées avec d'autres variables du jeu de données.

Variable tamhog

In [ ]:
sns.heatmap(corr_matrix.loc[corr_matrix['tamhog'].abs() > 0.95, corr_matrix['tamhog'].abs() > 0.95],
            annot=True, cmap = plt.cm.autumn_r, fmt='.3f');

La taille du foyer est représentée dans le jeu de données par plusieurs variables : tamhog, hhsize, hogar_total.
Nous n'allons garder qu'une seule variables parmis les variables ci-dessous.

In [ ]:
"""
r4t3, tamhog, hhsize et hogar_total sont des variables représentant le nombre de personnes dans le foyer.
Nous allons en garder seulement une variable : hhsize.
"""
head.drop(labels = ['tamhog', 'hogar_total', 'r4t3'], axis = 1, inplace = True)

Variable coopele

In [ ]:
sns.heatmap(corr_matrix.loc[corr_matrix['coopele'].abs() > 0.95, corr_matrix['coopele'].abs() > 0.95],
            annot=True, cmap = plt.cm.autumn_r, fmt='.3f');

Cette variable est très corrélée avec la variable "public" représentant un approvisionnement public en électricité. Nous n'allons pas supprimer dernière. Il servira par la suite à créer une nouvelle variable.

Variable area2

In [ ]:
sns.heatmap(corr_matrix.loc[corr_matrix['area2'].abs() > 0.95, corr_matrix['area2'].abs() > 0.95],
            annot=True, cmap = plt.cm.autumn_r, fmt='.3f');

In [ ]:
# La variable area1 est très corrélées avec la variable area2
head.drop(labels = 'area2', axis = 1, inplace = True)

Nous allons créer des variables ordinales (avec ordre) de niveau foyer à partir des variables de :
- Etat de l'état des murs, du toit et du sol
- Disponibilité des toilettes dans le foyer, à l'extérieur et/ou sans toilette
- Type d'approvisionnement en électricité
- Statut du foyer : Propriétaire ...etc

In [ ]:
# Etat des murs
head['epared_argmax'] = np.argmax(np.array(head[['epared1', 'epared2', 'epared3']]),
                           axis = 1)

In [ ]:
# Etat du toit
head['etecho_argmax'] = np.argmax(np.array(head[['etecho1', 'etecho2', 'etecho3']]),
                           axis = 1)

In [ ]:
# Etat du sol
head['eviv_argmax'] = np.argmax(np.array(head[['eviv1', 'eviv2', 'eviv3']]),
                           axis = 1)

In [ ]:
# Disponibilité de l'eau inside/outside/no water
head['abastagua_argmax'] = np.argmax(np.array(head[[ 'abastaguano',  'abastaguafuera','abastaguadentro']]),
                           axis = 1)

In [ ]:
# Connexion à un réseau élctriqueno elec/coopérative/public/privé
head['elec_argmax'] = np.argmax(np.array(head[['noelec','coopele', 'public', 'planpri']]),
                           axis = 1)

In [ ]:
# Propriétaire maison
head['tipovivi_argmax'] = np.argmax(np.array(head[['tipovivi5','tipovivi4', 'tipovivi3', 'tipovivi2','tipovivi1']]),
                           axis = 1)

La variable suivante permet de classer les foyers par ordre croissant selon s'ils disposent ou pas d'un réfrigirateur, un ordinateur et une télévision.

In [ ]:
head['electro_house'] = head['refrig'] + head['computer'] + head['television'] 

La variable suivante représente la différence entre le nombre de personnes du foyer et le nombre de personnes vivantes dans ce foyer

In [ ]:
head['diff_tamviv_hhsize'] = head['tamviv'] - head['hhsize']

*Suppression des variables utilisées ci-dessous pour la création de nouvelles variables*

In [ ]:
# Variables utilisées dans la création de nouvelles variables
head.drop(labels = [ 'epared1', 'epared2', 'epared3','etecho1', 'etecho2', 'etecho3',
                          'eviv1', 'eviv2', 'eviv3',
                          'abastaguano',  'abastaguafuera','abastaguadentro',
                          'noelec','coopele', 'public', 'planpri',
                          'refrig', 'computer','television',
                          'tipovivi5','tipovivi4', 'tipovivi3', 'tipovivi2','tipovivi1',
                          'tamviv']
                 , axis = 1, inplace = True)

In [ ]:
head.head()

### Analyse de niveau indivdidu

In [ ]:
# On ajoute les deux variables tamviv et hhsize pour calculer des moyennes par foyer
indiv = data[ids + ind_bool+ ind_ordered + target +['tamviv','hhsize']].copy()
indiv.shape

L'objectif de cette analyse est de créer, à partir des données individuelles, de nouvelles variables de niveau foyer (Agrégation par idhogar).

*Corrélation entre les variables*

In [ ]:
# Matrice de corrélation
corr_matrix = indiv.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.95)]

to_drop

Variable female

In [ ]:
sns.heatmap(corr_matrix.loc[corr_matrix['female'].abs() > 0.95, corr_matrix['female'].abs() > 0.95],
            annot=True, cmap = plt.cm.autumn_r, fmt='.3f');

Sans surprise la variable female est entièrement corrélée à la variable male.

In [ ]:
# La variable female est très corrélée avec male
indiv.drop(labels = 'male', axis = 1, inplace = True)

la variable suivante "instlevel_argmax" permet d'avoir le niveau max d'éducation de chaque individu.

In [ ]:
indiv['instlevel_argmax'] = np.argmax(np.array(indiv[['instlevel1', 'instlevel2','instlevel3','instlevel4','instlevel5','instlevel6','instlevel7','instlevel8','instlevel9']]),
                           axis = 1)

In [ ]:
indiv['indiv_digital'] = indiv['mobilephone'] + indiv['v18q']

In [ ]:
indiv.head()

In [ ]:
# Variables utilisées dans la création de nouvelles variables
indiv.drop(labels = ['instlevel1', 'instlevel2','instlevel3','instlevel4','instlevel5',
                           'instlevel6','instlevel7','instlevel8','instlevel9',
                    'mobilephone', 'v18q'
                    ]
                 , axis = 1, inplace = True)

Nous allons créer des variables ordinales (avec ordre) de niveau individu à partir des variables de :
- Niveau d'éducation
- ..

In [ ]:
indiv_pivot = pd.pivot_table(indiv,index=["idhogar"],
                            values=[
                                'age',
                                'rez_esc', 
                                'escolari',
                                'dis',
                                'female',
                                'instlevel_argmax',
                                'indiv_digital',
                                'tamviv',
                                'hhsize' 
                                   ],
                            aggfunc={
                             'age':np.sum,
                             'rez_esc':np.sum,
                             'escolari':np.sum, 
                             'dis' : np.sum,
                             'female' : np.sum,
                             'instlevel_argmax':[np.max,np.min,np.mean] , 
                             'indiv_digital':[np.max,np.min,np.mean] , 
                             'tamviv':[np.max,np.min] , 
                             'hhsize':[np.max,np.min]
                             
                },
                fill_value=0)

In [ ]:
indiv_toframe = pd.DataFrame(indiv_pivot.to_records())

In [ ]:
indiv_toframe.columns

In [ ]:
# Renommage des colonnes
indiv_toframe.columns = [hdr.replace("('", "").replace("', '", "_").replace("')", "") \
                     for hdr in indiv_toframe.columns]

*Création de variables de niveau foyer*

In [ ]:
# Age moyen par foyer
indiv_toframe['mean_age_hhsize'] = indiv_toframe['age_sum'] / indiv_toframe['hhsize_amax']
# Age moyen des personnes vivant ds la foyer
indiv_toframe['mean_age_tamviv'] = indiv_toframe['age_sum'] / indiv_toframe['tamviv_amax']

In [ ]:
# Nbre d'années d'éducation moyen par foyer
indiv_toframe['mean_escolari_hhsize'] = indiv_toframe['escolari_sum'] / indiv_toframe['hhsize_amax']
# Nbre d'années d'éducation moyen des personnes vivant ds la foyer
indiv_toframe['mean_escolari_tamviv'] = indiv_toframe['escolari_sum'] / indiv_toframe['tamviv_amax']

In [ ]:
# Nbre d'années d'éducation moyen par foyer
indiv_toframe['mean_rez_esc_hhsize'] = indiv_toframe['rez_esc_sum'] / indiv_toframe['hhsize_amax']
# Nbre d'années d'éducation moyen des personnes vivant ds la foyer
indiv_toframe['mean_rez_esc_tamviv'] = indiv_toframe['rez_esc_sum'] / indiv_toframe['tamviv_amax']

In [ ]:
# Nbre moyen de femmes par foyer
indiv_toframe['mean_female_hhsize'] = indiv_toframe['female_sum'] / indiv_toframe['hhsize_amax']
# NNbre moyen de femmes vivant ds la foyer
indiv_toframe['mean_female_tamviv'] = indiv_toframe['female_sum'] / indiv_toframe['tamviv_amax']

In [ ]:
# Nbre moyen de personnes handicapée par foyer
indiv_toframe['mean_dis_hhsize'] = indiv_toframe['dis_sum'] / indiv_toframe['hhsize_amax']
# Nbre moyen de personnes handicapéevivant ds la foyer
indiv_toframe['mean_dis_tamviv'] = indiv_toframe['dis_sum'] / indiv_toframe['tamviv_amax']

In [ ]:
indiv_toframe.head()

*Merge des deux dataframe head et indiv*

In [ ]:
head_agg = head.merge(indiv_toframe, on = 'idhogar',
                             how = 'left')

In [ ]:
head_agg.head()

In [ ]:
head_agg[head_agg['Target'].notnull()].shape

In [ ]:
head_agg[head_agg['Target'].isnull()].shape

# Modèles de prédiction

# Préparation des données d'apprentissage et de test

Le dataframe "head" est désormais prêt à être utilisé dans les modèles de prédiction.

Le jeu de données est déséquilibré, la classe 4 contient beaucoup plus d'observations que les autres classes.

Nous allons utiliser la méthode SMOTE pour rééquilibrer ce jeu de données.

Nous allons comparer les performances des deux modèles Random Forest et le gradient boosting sur les données brutes et les données après réequilibrage.

*Split des données brutes*

In [ ]:
# Labels for training
train_labels = np.array(list(head_agg[head_agg['Target'].notnull()]['Target'].astype(np.uint8)))

# Extract the training data
train_set = head_agg[head_agg['Target'].notnull()].drop(labels = ['Id', 'idhogar', 'Target'], axis = 1)
test_set = head_agg[head_agg['Target'].isnull()].drop(labels = ['Id', 'idhogar', 'Target'], axis = 1)

# Submission base which is used for making submissions to the competition
submission_base = df_test[['Id', 'idhogar']].copy()

*Réequilibrage des données*

In [ ]:
sm = SMOTE(random_state = 33)

In [ ]:
train_set_balanced, train_labels_balanced = sm.fit_sample(train_set, train_labels)

In [ ]:
train_set.shape
train_set_balanced.shape

In [ ]:
train_labels_balanced.shape

In [ ]:
train_labels.shape

In [ ]:
test_set.shape

In [ ]:
submission_base.shape

L'objet de l'étape suivante est d'imputer avec la mediane des valeurs manquantes puis de normaliser les variables.

In [ ]:
features = list(train_set.columns)
# Imputation avec la médiane

pipeline = Pipeline([('imputer', Imputer(strategy = 'median')), 
                      ('scaler', MinMaxScaler())])

# Fit and transform training data
train_set = pipeline.fit_transform(train_set)
test_set = pipeline.transform(test_set)

In [ ]:
# Fit and transform training balanced data
train_set_balanced = pipeline.fit_transform(train_set_balanced)

## Modèle de base : Random Forest

Modèle de Random forest avec 100 arbres et 10 k-fold en validation croisée

In [ ]:
model_no_smote = RandomForestClassifier(n_estimators=100, random_state=10, 
                               n_jobs = -1)
# 10 fold cross validation
cv_score_no_smote = cross_val_score(model_no_smote, train_set, train_labels, cv = 10, scoring = scorer)

print(f'10 Fold Cross Validation F1 Score = {round(cv_score_no_smote.mean(), 4)} with std = {round(cv_score_no_smote.std(), 4)}')

In [ ]:
model_smote = RandomForestClassifier(n_estimators=100, random_state=10, 
                               n_jobs = -1)
# 10 fold cross validation
cv_score_smote = cross_val_score(model_smote, train_set_balanced, train_labels_balanced, cv = 10, scoring = scorer)

print(f'10 Fold Cross Validation F1 Score = {round(cv_score_smote.mean(), 4)} with std = {round(cv_score_smote.std(), 4)}')

Après rééquilibrage des données, on passe d'un score F1 Macro de 0.3368 à 0.8824.
L'écart type à doublé mais il reste très acceptable.

*Variables les plus influentes (feature importance)*

In [ ]:
model_no_smote.fit(train_set, train_labels)

# Varibales importantes sur le jeu de données non rééquilibré
feature_importances_no_smote = pd.DataFrame({'feature': features, 'importance': model_no_smote.feature_importances_})
feature_importances_no_smote.head()

In [ ]:
model_smote.fit(train_set_balanced, train_labels_balanced)

# Varibales importantes sur le jeu de données rééquilibré
feature_importances_smote = pd.DataFrame({'feature': features, 'importance': model_smote.feature_importances_})
feature_importances_smote.head()

In [ ]:
def plot_feature_importances(df, n = 10, threshold = None):
    """Plots n most important features. Also plots the cumulative importance if
    threshold is specified and prints the number of features needed to reach threshold cumulative importance.
    Intended for use with any tree-based feature importances. 
    
    Args:
        df (dataframe): Dataframe of feature importances. Columns must be "feature" and "importance".
    
        n (int): Number of most important features to plot. Default is 15.
    
        threshold (float): Threshold for cumulative importance plot. If not provided, no plot is made. Default is None.
        
    Returns:
        df (dataframe): Dataframe ordered by feature importances with a normalized column (sums to 1) 
                        and a cumulative importance column
    
    Note:
    
        * Normalization in this case means sums to 1. 
        * Cumulative importance is calculated by summing features from most to least important
        * A threshold of 0.9 will show the most important features needed to reach 90% of cumulative importance
    
    """
    plt.style.use('fivethirtyeight')
    
    # Sort features with most important at the head
    df = df.sort_values('importance', ascending = False).reset_index(drop = True)
    
    # Normalize the feature importances to add up to one and calculate cumulative importance
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    df['cumulative_importance'] = np.cumsum(df['importance_normalized'])
    
    plt.rcParams['font.size'] = 12
    
    # Bar plot of n most important features
    df.loc[:n, :].plot.barh(y = 'importance_normalized', 
                            x = 'feature', color = 'darkgreen', 
                            edgecolor = 'k', figsize = (12, 8),
                            legend = False, linewidth = 2)

    plt.xlabel('Normalized Importance', size = 18); plt.ylabel(''); 
    plt.title(f'{n} Most Important Features', size = 18)
    plt.gca().invert_yaxis()
    
    
    if threshold:
        # Cumulative importance plot
        plt.figure(figsize = (8, 6))
        plt.plot(list(range(len(df))), df['cumulative_importance'], 'b-')
        plt.xlabel('Number of Features', size = 16); plt.ylabel('Cumulative Importance', size = 16); 
        plt.title('Cumulative Feature Importance', size = 18);
        
        # Number of features needed for threshold cumulative importance
        # This is the index (will need to add 1 for the actual number)
        importance_index = np.min(np.where(df['cumulative_importance'] > threshold))
        
        # Add vertical line to plot
        plt.vlines(importance_index + 1, ymin = 0, ymax = 1.05, linestyles = '--', colors = 'red')
        plt.show();
        
        print('{} features required for {:.0f}% of cumulative importance.'.format(importance_index + 1, 
                                                                                  100 * threshold))
    
    return df

In [ ]:
norm_fi_no_smote  = plot_feature_importances(feature_importances_no_smote, threshold=0.95)

In [ ]:
norm_fi_smote = plot_feature_importances(feature_importances_smote, threshold=0.95)

Les variables les plus importantes et comme on peut s'y attendre sont liées à l'éducation.


## Modèle : Light Gradient Boosting

In [ ]:
test_ids = list(head.loc[head['Target'].isnull(), 'idhogar'])

In [ ]:
def macro_f1_score(labels, predictions):
    # Reshape the predictions as needed
    predictions = predictions.reshape(len(np.unique(labels)), -1 ).argmax(axis = 0)
    
    metric_value = f1_score(labels, predictions, average = 'macro')
    
    # Return is name, value, is_higher_better
    return 'macro_f1', metric_value, True

In [ ]:

# Paramétres pour un test
params = {'boosting_type': 'dart', 
                  'colsample_bytree': 0.88, 
                  'learning_rate': 0.028, 
                   'min_child_samples': 10, 
                   'num_leaves': 36, 'reg_alpha': 0.76, 
                   'reg_lambda': 0.43, 
                   'subsample_for_bin': 40000, 
                   'subsample': 0.54, 
                   'class_weight': 'balanced'}
    
# Modèles
model = lgb.LGBMClassifier(**params, objective = 'multiclass', 
                               n_jobs = -1, n_estimators = 10000,
                               random_state = 10)
    
# Using stratified kfold cross validation
strkfold = StratifiedKFold(n_splits = 5, shuffle = True)

In [ ]:
strkfold.get_n_splits(train_set_balanced, train_labels_balanced)

In [ ]:
for train_index, test_index in strkfold.split(train_set, train_labels):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = train_set[train_index], train_set[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

In [ ]:
for train_index, test_index in strkfold.split(train_set_balanced, train_labels_balanced):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_balanced, X_test_balanced = train_set_balanced[train_index], train_set_balanced[test_index]
    y_train_balanced, y_test_balanced = train_labels_balanced[train_index], train_labels_balanced[test_index]

In [ ]:
model_lgb = model.fit(X_train, y_train, early_stopping_rounds =20, 
                  eval_metric = macro_f1_score,
                  eval_set = [(X_train, y_train), (X_test, y_test)],
                  eval_names = ['train', 'valid'],
                  verbose = 200)

In [ ]:
predictions = model_lgb.predict(test_set)
predictions = pd.DataFrame({'idhogar': test_ids,
                            'Target': predictions})

     # Make a submission dataframe
submission = submission_base.merge(predictions, 
                                       on = 'idhogar',
                                       how = 'left').drop(labels = ['idhogar'], axis = 1)
    
    # Fill in households missing a head
lgb_submission = submission['Target'] = submission['Target'].fillna(4).astype(np.int8)


lgb_submission.to_csv('lgb_submission.csv', index = False)

In [ ]:
model_lgb_balanced = model.fit(X_train_balanced, y_train_balanced, early_stopping_rounds = 20, 
                  eval_metric = macro_f1_score,
                  eval_set = [(X_train_balanced, y_train_balanced), (X_test_balanced, y_test_balanced)],
                  eval_names = ['train_balanced', 'valid_balanced'],
                  verbose = 200)

In [ ]:
predictions = model_lgb_balanced.predict(test_set)
predictions = pd.DataFrame({'idhogar': test_ids,
                            'Target': predictions})

     # Make a submission dataframe
submission = submission_base.merge(predictions, 
                                       on = 'idhogar',
                                       how = 'left').drop(labels = ['idhogar'], axis = 1)
    
    # Fill in households missing a head
lgb_balanced_submission = submission['Target'] = submission['Target'].fillna(4).astype(np.int8)


lgb_balanced_submission.to_csv('lgb_submission.csv', index = False)

In [ ]:
lgb_balanced_submission.shape

##  Prédiction

In [ ]:
test_ids = list(head.loc[head['Target'].isnull(), 'idhogar'])

In [ ]:
len(test_ids)

In [ ]:
def submit(model, train, train_labels, test, test_ids):
    """Train and test a model on the dataset"""
    
    # Train on the data
    model.fit(train, train_labels)
    predictions = model.predict(test)
    predictions = pd.DataFrame({'idhogar': test_ids,
                               'Target': predictions})

     # Make a submission dataframe
    submission = submission_base.merge(predictions, 
                                       on = 'idhogar',
                                       how = 'left').drop(labels = ['idhogar'], axis = 1)
    
    # Fill in households missing a head
    submission['Target'] = submission['Target'].fillna(4).astype(np.int8)

    return submission

In [ ]:
rf_submission = submit(RandomForestClassifier(n_estimators = 100, 
                                              random_state=10, n_jobs = -1), 
                         train_set, train_labels, test_set, test_ids)

rf_submission.to_csv('rf_submission.csv', index = False)

In [ ]:
rf_submission.shape

In [ ]:
rf_balanced_submission = submit(RandomForestClassifier(n_estimators = 100, 
                                              random_state=10, n_jobs = -1), 
                         train_set_balanced, train_labels_balanced, test_set, test_ids)

rf_balanced_submission.to_csv('rf_balanced_submission.csv', index = False)

In [ ]:
rf_balanced_submission.shape